# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


In [13]:
data= pd.read_csv("weather_data.csv")
data.head()
data = data.rename(columns={"Max Temp": "Temp", "Wind Speed": "Wind_Speed"})
data["Cloudiness"] = pd.to_numeric(data["Cloudiness"])
data = data.astype({"Wind_Speed": int})
data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind_Speed
0,Yellowknife,90,CA,1580513509,76,62.46,-114.35,-20.00,4
1,Hithadhoo,59,MV,1580563519,72,-0.60,73.08,27.57,4
2,Hobart,75,AU,1580549793,65,-42.88,147.33,26.00,5
3,Tasiilaq,100,GL,1580493544,45,65.61,-37.64,-3.00,14
4,Udachny,75,RU,1580543595,82,66.42,112.40,-27.00,0


In [14]:
gmaps.configure(api_key=g_key)
locations = data[["Lat", "Lng"]]
humidity = data["Humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=5)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
data = data[(data.Temp > 22) &(data.Temp <26)]
data = data[(data.Cloudiness > 20) & (data.Cloudiness < 50)]
data = data.dropna()
data


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind_Speed
112,Chui,48,UY,1580510397,64,-33.70,-53.46,22.99,5
140,Ambovombe,40,MG,1580571997,83,-25.17,46.08,22.28,5
196,Celestun,33,MX,1580514667,88,20.87,-90.40,24.36,5
206,Emerald,40,AU,1580547350,83,-23.53,148.17,25.00,2
224,Coquimbo,40,CL,1580514274,72,-29.95,-71.34,25.56,5
230,San Sebastian Tenango,40,MX,1580516722,28,18.65,-98.18,23.33,10
252,Mahabalipuram,40,IN,1580560816,83,12.63,80.17,25.00,1
278,Marsh Harbour,41,BS,1580510959,67,26.54,-77.06,23.25,4
415,Auchi,48,NG,1580492563,13,7.07,6.27,25.71,0
417,Tondano,40,ID,1580550936,94,1.30,124.91,24.00,0


In [6]:
data["Hotel Name"] = ""
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "name": "hotel",
    "key": g_key,}

for index, row in data.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name = requests.get(base_url, params=params)
    name = name.json()
    try:
        data.loc[index, "Hotel Name"] = name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
data


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temp,Wind_Speed,Hotel Name
112,Chui,48,UY,1580510397,64,-33.70,-53.46,22.99,5,Gaboto
140,Ambovombe,40,MG,1580571997,83,-25.17,46.08,22.28,5,Hotel Le Source
196,Celestun,33,MX,1580514667,88,20.87,-90.40,24.36,5,Playa 55 beach escape
206,Emerald,40,AU,1580547350,83,-23.53,148.17,25.00,2,Emerald Executive Apartments
224,Coquimbo,40,CL,1580514274,72,-29.95,-71.34,25.56,5,El Hotel Iberia
230,San Sebastian Tenango,40,MX,1580516722,28,18.65,-98.18,23.33,10,Hotel Posada Rancho Los Laureles
252,Mahabalipuram,40,IN,1580560816,83,12.63,80.17,25.00,1,Four Points by Sheraton Mahabalipuram Resort &...
278,Marsh Harbour,41,BS,1580510959,67,26.54,-77.06,23.25,4,Lofty Fig Villas
415,Auchi,48,NG,1580492563,13,7.07,6.27,25.71,0,Presidential Hotel Auchi
417,Tondano,40,ID,1580550936,94,1.30,124.91,24.00,0,YAMA RESORT


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in data.iterrows()]
locations = data[["Lat", "Lng"]]

In [9]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))